In [1]:
from func_def import *
import keras

In [2]:
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
7192,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7193,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7194,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [3]:
#tester unit
from func_def import *
NAME = 'Max Holloway'
df[(df.fighter==NAME)|(df.opponent==NAME)][0:3]

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
45,http://www.ufcstats.com/fight-details/96704d7b...,http://www.ufcstats.com/event-details/89a40703...,2023-08-26,W,Max Holloway,Chan Sung Jung,Featherweight Bout,KO/TKO,3,0:23,5 Rnd (5-5-5-5-5),34of77,31of35,10of16,71of124,1of1,3of3,25of110,6of8,3of6,34of122,0of2,0of0,0.45,0.41,0.13,0.94,0.01,0.04,0.73,0.17,0.08,1.00,0.00,0.0
265,http://www.ufcstats.com/fight-details/6049301b...,http://www.ufcstats.com/event-details/15b1b21c...,2023-04-15,W,Max Holloway,Arnold Allen,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),77of180,42of48,28of34,146of261,1of1,0of0,48of173,18of36,10of16,76of225,0of0,0of0,0.52,0.28,0.19,0.99,0.00,0.00,0.63,0.23,0.13,1.00,0.00,0.0
658,http://www.ufcstats.com/fight-details/4047e981...,http://www.ufcstats.com/event-details/4a9e3056...,2022-07-02,W,Alexander Volkanovski,Max Holloway,UFC Featherweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),152of316,19of24,28of32,175of340,24of32,0of0,64of213,42of59,21of29,109of283,18of18,0of0,0.76,0.09,0.14,0.87,0.12,0.00,0.50,0.33,0.16,0.85,0.14,0.0


In [7]:
#test
get_last_n_stat_mean(fighter=NAME,stat_col='sig_str_head_perc',time='2023-08-26',data=df,n=5)

0.5239999999999999

In [3]:
# Pick features
FEATURES = ['head_sig_str','body_sig_str','leg_sig_str','distance_sig_str','clinch_sig_str','ground_sig_str']

feature_columns = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(feature_columns)

['f_head_sig_str', 'f_body_sig_str', 'f_leg_sig_str', 'f_distance_sig_str', 'f_clinch_sig_str', 'f_ground_sig_str', 'o_head_sig_str', 'o_body_sig_str', 'o_leg_sig_str', 'o_distance_sig_str', 'o_clinch_sig_str', 'o_ground_sig_str']


In [4]:
data = df[['date','fighter','opponent','result'] + feature_columns].copy()

for prefix in ['f_','o_']:
    for feature in FEATURES:
        data[f'{prefix}{feature}'] = pd.to_numeric([int(item.split('of')[0]) for item in df[f'{prefix}{feature}']])
        data[f'{prefix}{feature}_att'] = pd.to_numeric([int(item.split('of')[1]) for item in df[f'{prefix}{feature}']])

data.tail(3)

,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att
7216,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,17,23,0,6,8,4,14,4,0,37,24,16,39,38,0,20,9,13,35,6,1
7217,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,0,1,3,3,2,2,4,1,2,3,2,1,1,1,4,4,2,2,5,1,2
7218,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,39,0,0,43,20,63,116,10,0,241,12,6,258,1,0,127,22,72,209,12,0


In [6]:
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]
for stat in STATS:
    for index in tqdm(data.index):
        data.loc[index,f'f_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'o_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data)

data.tail(3)

100%|██████████| 7219/7219 [00:55<00:00, 129.42it/s]


,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_distance_sig_str_mean,o_distance_sig_str_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean
7216,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,17,23,0,6,8,4,14,4,0,37,24,16,39,38,0,20,9,13,35,6,1,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.000000,69.166667,0.000000,6.0,0.0,4.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667,0.000000,136.666667,0.0,8.166667,0.0,5.333333
7217,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,0,1,3,3,2,2,4,1,2,3,2,1,1,1,4,4,2,2,5,1,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,42.000000,39.500000,4.000000,5.5,12.0,2.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000,79.000000,72.500000,5.0,7.000000,12.0,3.000000
7218,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,39,0,0,43,20,63,116,10,0,241,12,6,258,1,0,127,22,72,209,12,0,34.333333,0.000000,6.0,0.0,3.666667,0.000000,40.333333,0.000000,3.666667,0.0,0.0,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000,89.666667,0.000000,5.0,0.000000,0.0,0.000000


In [7]:
for feature in FEATURES:
    data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']
    data[f'{feature}_att_dif'] = data[f'f_{feature}_att'] - data[f'o_{feature}_att']
    data[f'{feature}_mean_dif'] = data[f'f_{feature}_mean'] - data[f'o_{feature}_mean']
    data[f'{feature}_att_mean_dif'] = data[f'f_{feature}_att_mean'] - data[f'o_{feature}_att_mean']

data.head(3)

,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_distance_sig_str_mean,o_distance_sig_str_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif
0,2023-08-26,Max Holloway,Chan Sung Jung,W,34,31,10,71,1,3,25,6,3,34,0,0,77,35,16,124,1,3,110,8,6,122,2,0,78.407407,39.272727,26.777778,4.909091,10.444444,4.363636,100.333333,37.909091,9.037037,2.909091,6.259259,7.727273,194.592593,102.090909,35.703704,6.818182,13.629630,5.818182,223.481481,101.181818,11.851852,3.454545,8.592593,10.090909,9,-33,39.134680,92.501684,25,27,21.868687,28.885522,7,10,6.080808,7.811448,37,2,62.424242,122.299663,1,-1,6.127946,8.397306,3,3,-1.468013,-1.498316
1,2023-08-26,Anthony Smith,Ryan Spann,W,44,12,35,90,1,0,39,14,6,50,0,9,92,15,40,146,1,0,158,23,6,174,0,13,21.052632,14.500000,6.210526,2.900000,4.789474,0.100000,26.947368,11.700000,3.157895,1.900000,1.947368,3.900000,51.473684,35.800000,9.842105,3.300000,5.368421,0.200000,59.631579,31.600000,4.421053,2.600000,2.631579,5.100000,5,-66,6.552632,15.673684,-2,-8,3.310526,6.542105,29,34,4.689474,5.168421,40,-28,15.247368,28.031579,1,1,1.257895,1.821053,-9,-13,-1.952632,-2.468421
2,2023-08-26,Giga Chikadze,Alex Caceres,W,25,22,18,65,0,0,25,25,12,62,0,0,83,38,20,139,2,0,110,44,14,168,0,0,33.500000,31.500000,12.750000,11.714286,7.625000,5.892857,47.875000,41.607143,0.375000,4.464286,5.625000,3.035714,100.000000,72.714286,19.000000,15.607143,9.875000,7.214286,119.125000,84.964286,1.250000,6.071429,8.500000,4.500000,0,-27,2.000000,27.285714,-3,-6,1.035714,3.392857,6,6,1.732143,2.660714,3,-29,6.267857,34.160714,0,2,-4.089286,-4.821429,0,0,2.589286,4.000000


In [8]:
X = [f'{feature}_mean_dif' for feature in FEATURES] + [f'{feature}_att_mean_dif' for feature in FEATURES]
y = 'result'
X

['head_sig_str_mean_dif',
 'body_sig_str_mean_dif',
 'leg_sig_str_mean_dif',
 'distance_sig_str_mean_dif',
 'clinch_sig_str_mean_dif',
 'ground_sig_str_mean_dif',
 'head_sig_str_att_mean_dif',
 'body_sig_str_att_mean_dif',
 'leg_sig_str_att_mean_dif',
 'distance_sig_str_att_mean_dif',
 'clinch_sig_str_att_mean_dif',
 'ground_sig_str_att_mean_dif']

In [9]:
set_train_test(X_columns=X, y_columns=y, data=data, test_size=0.3, valid_size=0)
data.tail(3)

Data split: Data=(7219, 77), train_set=(5060, 77), validation_set=(0, 77), test_set=(2159, 77)


,set,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_distance_sig_str_mean,o_distance_sig_str_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif
7216,train,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,17,23,0,6,8,4,14,4,0,37,24,16,39,38,0,20,9,13,35,6,1,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.000000,69.166667,0.000000,6.0,0.0,4.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667,0.000000,136.666667,0.0,8.166667,0.0,5.333333,6,17,-26.166667,-71.333333,9,15,-27.0,-45.666667,7,3,-26.333333,-33.166667,3,4,-69.166667,-136.666667,19,32,-6.000000,-8.166667,0,-1,-4.333333,-5.333333
7217,train,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,0,1,3,3,2,2,4,1,2,3,2,1,1,1,4,4,2,2,5,1,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,42.000000,39.500000,4.000000,5.5,12.0,2.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000,79.000000,72.500000,5.0,7.000000,12.0,3.000000,-1,-1,12.500000,16.500000,0,0,-3.5,-4.500000,-2,-1,1.500000,1.500000,-4,-4,2.500000,6.500000,0,0,-1.500000,-2.000000,1,2,9.500000,9.000000
7218,train,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,39,0,0,43,20,63,116,10,0,241,12,6,258,1,0,127,22,72,209,12,0,34.333333,0.000000,6.0,0.0,3.666667,0.000000,40.333333,0.000000,3.666667,0.0,0.0,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000,89.666667,0.000000,5.0,0.000000,0.0,0.000000,-18,114,34.333333,84.000000,-12,-10,6.0,7.000000,-57,-66,3.666667,3.666667,-77,49,40.333333,89.666667,-10,-11,3.666667,5.000000,0,0,0.000000,0.000000


### Logistic regression model train

In [10]:
data.columns

Index(['set', 'date', 'fighter', 'opponent', 'result', 'f_head_sig_str',
       'f_body_sig_str', 'f_leg_sig_str', 'f_distance_sig_str',
       'f_clinch_sig_str', 'f_ground_sig_str', 'o_head_sig_str',
       'o_body_sig_str', 'o_leg_sig_str', 'o_distance_sig_str',
       'o_clinch_sig_str', 'o_ground_sig_str', 'f_head_sig_str_att',
       'f_body_sig_str_att', 'f_leg_sig_str_att', 'f_distance_sig_str_att',
       'f_clinch_sig_str_att', 'f_ground_sig_str_att', 'o_head_sig_str_att',
       'o_body_sig_str_att', 'o_leg_sig_str_att', 'o_distance_sig_str_att',
       'o_clinch_sig_str_att', 'o_ground_sig_str_att', 'f_head_sig_str_mean',
       'o_head_sig_str_mean', 'f_body_sig_str_mean', 'o_body_sig_str_mean',
       'f_leg_sig_str_mean', 'o_leg_sig_str_mean', 'f_distance_sig_str_mean',
       'o_distance_sig_str_mean', 'f_clinch_sig_str_mean',
       'o_clinch_sig_str_mean', 'f_ground_sig_str_mean',
       'o_ground_sig_str_mean', 'f_head_sig_str_att_mean',
       'o_head_sig_str_att_me

In [11]:
X

['head_sig_str_mean_dif',
 'body_sig_str_mean_dif',
 'leg_sig_str_mean_dif',
 'distance_sig_str_mean_dif',
 'clinch_sig_str_mean_dif',
 'ground_sig_str_mean_dif',
 'head_sig_str_att_mean_dif',
 'body_sig_str_att_mean_dif',
 'leg_sig_str_att_mean_dif',
 'distance_sig_str_att_mean_dif',
 'clinch_sig_str_att_mean_dif',
 'ground_sig_str_att_mean_dif']

In [12]:
# train model
X_train = data.loc[data['set'] == 'train', X].values
y_train = data.loc[data['set'] == 'train', y].values
    
# Create a logistic regression model
model = LogisticRegression(C=0.1)

# Fit the model to the training data
model.fit(X_train, y_train)
model

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1)

In [13]:
# make predictions
data[f'{y}_pred'] = model.predict(data[X])
data[f'{y}_pred_proba'] = [max(item) for item in model.predict_proba(data[X])] #get max probability 
data.tail(3)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,set,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_distance_sig_str_mean,o_distance_sig_str_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif,result_pred,result_pred_proba
7216,train,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,17,23,0,6,8,4,14,4,0,37,24,16,39,38,0,20,9,13,35,6,1,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.000000,69.166667,0.000000,6.0,0.0,4.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667,0.000000,136.666667,0.0,8.166667,0.0,5.333333,6,17,-26.166667,-71.333333,9,15,-27.0,-45.666667,7,3,-26.333333,-33.166667,3,4,-69.166667,-136.666667,19,32,-6.000000,-8.166667,0,-1,-4.333333,-5.333333,W,0.607267
7217,train,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,0,1,3,3,2,2,4,1,2,3,2,1,1,1,4,4,2,2,5,1,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,42.000000,39.500000,4.000000,5.5,12.0,2.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000,79.000000,72.500000,5.0,7.000000,12.0,3.000000,-1,-1,12.500000,16.500000,0,0,-3.5,-4.500000,-2,-1,1.500000,1.500000,-4,-4,2.500000,6.500000,0,0,-1.500000,-2.000000,1,2,9.500000,9.000000,W,0.672990
7218,train,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,39,0,0,43,20,63,116,10,0,241,12,6,258,1,0,127,22,72,209,12,0,34.333333,0.000000,6.0,0.0,3.666667,0.000000,40.333333,0.000000,3.666667,0.0,0.0,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000,89.666667,0.000000,5.0,0.000000,0.0,0.000000,-18,114,34.333333,84.000000,-12,-10,6.0,7.000000,-57,-66,3.666667,3.666667,-77,49,40.333333,89.666667,-10,-11,3.666667,5.000000,0,0,0.000000,0.000000,W,0.671145


In [14]:
# Evaluate model
test_data = data[data.set=='test']
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.6484483557202408


In [15]:
# Evaluate high probability predictions only
HIGH_PROBABILITY_SCORE = 0.8
test_data = data[(data.set=='test')&(data.result_pred_proba > HIGH_PROBABILITY_SCORE)]
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.9
